In [1]:
#import the dataset 
from keras.datasets import fashion_mnist

In [2]:
#import the required libraries 
from matplotlib import pyplot
import numpy as np

In [3]:
#load the data into train and test 
(X_train,y_train),(X_test,y_test)=fashion_mnist.load_data()
#pyplot.imshow(X_train[2])

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
noOfImages = X_train.shape[0]
X_train = (1.0/255)*np.array([X_train[i].flatten() for i in range(0,noOfImages)])
X_train = np.array([X_train[i].flatten() for i in range(0,noOfImages)])
noOftestImages= X_test.shape[0]
X_test =(1.0/255)*np.array([X_test[i].flatten() for i in range(0,noOftestImages)])
X_test =np.array([X_test[i].flatten() for i in range(0,noOftestImages)])
print(X_test.shape)


(10000, 784)


In [5]:
print(X_train[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.00392157 0.         0.         0.05098039 0.28627451 

In [6]:
def softmax(X):
  X=np.exp(X)
  sum=np.sum(X,axis=0)
  return X/sum 


In [7]:
def sigmoidFunc(X):

  return 1.0/(1.0+np.exp(-X))
  #return res

In [8]:
def gDash(X):
  return sigmoidFunc(X)*(1-sigmoidFunc(X))

In [9]:
def forwardPropagation(X,parameters):
  preactivation={}
  activation={}
  activation['h0']=X.T
 
  for k in range(1,noOfHiddenLayers+1):
    preactivation['a'+str(k)]=np.dot(parameters['W'+str(k)],activation['h'+str(k-1)])+parameters['b'+str(k)]
    activation['h'+str(k)]=sigmoidFunc(preactivation['a'+str(k)])
 
  preactivation['a'+str(noOfHiddenLayers+1)]=np.dot(parameters['W'+str(noOfHiddenLayers+1)],activation['h'+str(noOfHiddenLayers)])+parameters['b'+str(noOfHiddenLayers+1)]
  y=softmax(preactivation['a'+str(noOfHiddenLayers+1)])    

  return (preactivation,activation,y)



In [10]:
def backPropagation(parameters,activation,preactivation,yhat,X,y_train):
  grads={}
  eIndicator=np.zeros((10,X.shape[0]))
  eIndicator[y_train,np.arange(X.shape[0])]=1

  grads['a'+str(noOfHiddenLayers+1)]= -(eIndicator-yhat)

  for j in range(noOfHiddenLayers+1,0,-1):
    grads['W'+str(j)]= np.dot(grads['a'+str(j)],activation['h'+str(j-1)].T)
 
    grads['b'+str(j)]= np.sum(grads['a'+str(j)],axis=1,keepdims=True)
    
    grads['h'+str(j-1)]=np.dot(parameters['W'+str(j)].T,grads['a'+str(j)])
    
    if (j!=1):
      grads['a'+str(j-1)]=grads['h'+str(j-1)]*gDash(preactivation['a'+str(j-1)])
  return grads


In [11]:
def Loss(yhat,y_train,X):
  eIndicator=np.zeros((10,X.shape[0]))
  eIndicator[y_train,np.arange(X.shape[0])]=1
  eIndicator=eIndicator*yhat
  eIndicator=eIndicator.sum(axis=0)
  eIndicator=np.log(eIndicator)
  return -sum(eIndicator)


In [24]:

l=10 #output classes
noOfneuronsEach=[120,20]
noOfHiddenLayers=len(noOfneuronsEach)
inputNeuronSize=X_train.shape[1]
print(inputNeuronSize)
parameters={}
batchSize=600
iterations=100
#eta=1   
eta=0.001   #for rmsprop higher learning rate gradients explode causing overflow
eta=0.002 #Nadam
#input W
#initialization
parameters['W'+str(1)] =np.random.uniform(low=-0.5,high=0.5,size=(noOfneuronsEach[0], inputNeuronSize))
parameters['b'+str(1)]= np.random.uniform(low=-0.5,high=0.5,size=(noOfneuronsEach[0],1))
#parameters['W'+str(1)] =0+1.5*np.random.randn(noOfneuronsEach[0], inputNeuronSize)
#parameters['b'+str(1)]= 0+1.5*np.random.randn(noOfneuronsEach[0],1)
for i in range(2,noOfHiddenLayers+1):
  parameters['W'+str(i)] = np.random.uniform(low=-0.5,high=0.5,size=(noOfneuronsEach[i-1], noOfneuronsEach[i-2]))
  parameters['b'+str(i)]= np.random.uniform(low=-0.5,high=0.5,size=(noOfneuronsEach[i-1],1))
  #parameters['W'+str(i)] = 0+1.5*np.random.randn(noOfneuronsEach[i-1], noOfneuronsEach[i-2])
  #parameters['b'+str(i)]= 0+1.5*np.random.randn(noOfneuronsEach[i-1],1)
#Output W
parameters['W'+str(noOfHiddenLayers+1)] = np.random.uniform(low=-0.5,high=0.5,size=(l, noOfneuronsEach[-1]))
parameters['b'+str(noOfHiddenLayers+1)]= np.random.uniform(low=-0.5,high=0.5,size=(l,1))
#parameters['W'+str(noOfHiddenLayers+1)] = 0+1.5*np.random.randn(l, noOfneuronsEach[-1])
#parameters['b'+str(noOfHiddenLayers+1)]= 0+1.5*np.random.randn(l,1)


784


In [70]:
def vanillagradDescent():
  t=0
  while(t < iterations):
    print("Epoch ",t)
    #print(parameters)
    mini=0
    while(mini<(noOfImages/batchSize)):
      #print("Batch ",mini)
      X_mini=X_train[(mini*batchSize):((mini+1)*batchSize-1)]
      y_mini=y_train[(mini*batchSize):((mini+1)*batchSize-1)]
      preactivation,activation,yhat=forwardPropagation(X_mini,parameters)
      gradients=backPropagation(parameters,activation,preactivation,yhat,X_mini,y_mini)
    #print("gradients",gradients)
      for i in range(1,noOfHiddenLayers+2):
        parameters['W'+str(i)] -=  eta*(1.0/X_mini.shape[0])*gradients['W'+str(i)]
        parameters['b'+str(i)] -= eta*(1.0/X_mini.shape[0])*gradients['b'+str(i)]
      mini+=1
    _,_,yhat=forwardPropagation(X_train,parameters)
    print("loss after epoch ",Loss(yhat,y_train,X_train))
    
    t+=1
vanillagradDescent()


Epoch  0
loss after epoch  47573.393426219605
Epoch  1
loss after epoch  41904.39835775272
Epoch  2
loss after epoch  33197.24226676189
Epoch  3
loss after epoch  30161.114459625456
Epoch  4


KeyboardInterrupt: ignored

In [85]:
def momentumGD():
  #initialization
  prevW={}
  prevb={}
  gamma=0.9
  prevW['W'+str(1)] =np.zeros((noOfneuronsEach[0], inputNeuronSize))
  prevb['b'+str(1)]= np.zeros((noOfneuronsEach[0],1))
  for i in range(2,noOfHiddenLayers+1):
    prevW['W'+str(i)] = np.zeros((noOfneuronsEach[i-1], noOfneuronsEach[i-2]))
    prevb['b'+str(i)]= np.zeros((noOfneuronsEach[i-1],1))
  prevW['W'+str(noOfHiddenLayers+1)] = np.zeros((l, noOfneuronsEach[-1]))
  prevb['b'+str(noOfHiddenLayers+1)]= np.zeros((l,1))
  t=0
  while(t<iterations):
    print("Epoch ",t)
    mini=0
    while(mini<(noOfImages/batchSize)):
      #print("Batch ",mini)
      X_mini=X_train[(mini*batchSize):((mini+1)*batchSize-1)]
      y_mini=y_train[(mini*batchSize):((mini+1)*batchSize-1)]
      preactivation,activation,yhat=forwardPropagation(X_mini,parameters)
      gradients=backPropagation(parameters,activation,preactivation,yhat,X_mini,y_mini)
      #print("gradients",gradients)
      for i in range(1,noOfHiddenLayers+2):
        w=gamma*prevW['W'+str(i)]+eta*(1.0/X_mini.shape[0])*gradients['W'+str(i)]
        b=gamma*prevb['b'+str(i)]+eta*(1.0/X_mini.shape[0])*gradients['b'+str(i)]
        parameters['W'+str(i)] -=  w
        parameters['b'+str(i)] -=  b
        prevW['W'+str(i)]=w
        prevb['b'+str(i)]=b
      mini+=1
    _,_,yhat=forwardPropagation(X_train,parameters)
    print("loss after epoch ",Loss(yhat,y_train,X_train))
    
    t+=1
momentumGD()



Epoch  0
loss after epoch  139104.58233516416
Epoch  1
loss after epoch  135653.34137150904
Epoch  2
loss after epoch  133530.29032065018
Epoch  3
loss after epoch  131507.08960641167
Epoch  4
loss after epoch  129349.62693123208
Epoch  5
loss after epoch  127035.64020326082
Epoch  6
loss after epoch  124597.91580561991
Epoch  7
loss after epoch  122062.11478591064
Epoch  8
loss after epoch  119437.66889407003
Epoch  9
loss after epoch  116732.94189523862
Epoch  10
loss after epoch  113966.5784082077
Epoch  11
loss after epoch  111173.31822416405
Epoch  12
loss after epoch  108400.25914087553
Epoch  13
loss after epoch  105690.34601707169
Epoch  14
loss after epoch  103070.88501845609
Epoch  15
loss after epoch  100556.71147232091
Epoch  16
loss after epoch  98155.97612306397
Epoch  17
loss after epoch  95871.9689163332
Epoch  18
loss after epoch  93703.44196097103
Epoch  19
loss after epoch  91645.66990811274
Epoch  20
loss after epoch  89692.0732574155
Epoch  21


KeyboardInterrupt: ignored

In [86]:
def rmsProp():
  #initialization
  prevW={}
  prevb={}
  epsilon=1e-8
  beta=0.9
  prevW['W'+str(1)] =np.zeros((noOfneuronsEach[0], inputNeuronSize))
  prevb['b'+str(1)]= np.zeros((noOfneuronsEach[0],1))
  for i in range(2,noOfHiddenLayers+1):
    prevW['W'+str(i)] = np.zeros((noOfneuronsEach[i-1], noOfneuronsEach[i-2]))
    prevb['b'+str(i)]= np.zeros((noOfneuronsEach[i-1],1))
  prevW['W'+str(noOfHiddenLayers+1)] = np.zeros((l, noOfneuronsEach[-1]))
  prevb['b'+str(noOfHiddenLayers+1)]= np.zeros((l,1))
  t=0
  while(t<iterations):
    print("Epoch ",t)
    mini=0
    while(mini<(noOfImages/batchSize)):
      #print("Batch ",mini)
      X_mini=X_train[(mini*batchSize):((mini+1)*batchSize-1)]
      y_mini=y_train[(mini*batchSize):((mini+1)*batchSize-1)]
      preactivation,activation,yhat=forwardPropagation(X_mini,parameters)
      gradients=backPropagation(parameters,activation,preactivation,yhat,X_mini,y_mini)
      #print("gradients",gradients)
      for i in range(1,noOfHiddenLayers+2):
        prevW['W'+str(i)]= beta*prevW['W'+str(i)]+(1.0 - beta)* (gradients['W'+str(i)]**2)
        prevb['b'+str(i)]= beta*prevb['b'+str(i)]+(1.0 - beta)* (gradients['b'+str(i)]**2)
        
        parameters['W'+str(i)] -=  ((eta/np.sqrt(prevW['W'+str(i)]+epsilon))*gradients['W'+str(i)])
        parameters['b'+str(i)] -=  ((eta/np.sqrt(prevb['b'+str(i)]+epsilon))*gradients['b'+str(i)])
      mini+=1
    _,_,yhat=forwardPropagation(X_train,parameters)
    print("loss after epoch ",Loss(yhat,y_train,X_train))
    
    t+=1
rmsProp()

Epoch  0
loss after epoch  51233.51974791119
Epoch  1
loss after epoch  39667.038907740905
Epoch  2


KeyboardInterrupt: ignored

In [25]:
def nadam():
  #initialization
  prevW={}
  prevb={}
  mW={}
  mb={}
  epsilon=1e-8
  beta1=0.9
  beta2=0.999
  prevW['W'+str(1)] =np.zeros((noOfneuronsEach[0], inputNeuronSize))
  prevb['b'+str(1)]= np.zeros((noOfneuronsEach[0],1))
  for i in range(2,noOfHiddenLayers+1):
    prevW['W'+str(i)] = np.zeros((noOfneuronsEach[i-1], noOfneuronsEach[i-2]))
    prevb['b'+str(i)]= np.zeros((noOfneuronsEach[i-1],1))
  prevW['W'+str(noOfHiddenLayers+1)] = np.zeros((l, noOfneuronsEach[-1]))
  prevb['b'+str(noOfHiddenLayers+1)]= np.zeros((l,1))

  mW['W'+str(1)] =np.zeros((noOfneuronsEach[0], inputNeuronSize))
  mb['b'+str(1)]= np.zeros((noOfneuronsEach[0],1))
  for i in range(2,noOfHiddenLayers+1):
    mW['W'+str(i)] = np.zeros((noOfneuronsEach[i-1], noOfneuronsEach[i-2]))
    mb['b'+str(i)]= np.zeros((noOfneuronsEach[i-1],1))
  mW['W'+str(noOfHiddenLayers+1)] = np.zeros((l, noOfneuronsEach[-1]))
  mb['b'+str(noOfHiddenLayers+1)]= np.zeros((l,1))


  t=0 #iterations 
  f=0 # update number 
  while(t<iterations):
    print("Epoch ",t)
    mini=0
    while(mini<(noOfImages/batchSize)):
      #print("Batch ",mini)
      X_mini=X_train[(mini*batchSize):((mini+1)*batchSize-1)]
      y_mini=y_train[(mini*batchSize):((mini+1)*batchSize-1)]
      preactivation,activation,yhat=forwardPropagation(X_mini,parameters)
      gradients=backPropagation(parameters,activation,preactivation,yhat,X_mini,y_mini)
      f+=1
      for i in range(1,noOfHiddenLayers+2):
        mW['W'+str(i)]= beta1*mW['W'+str(i)]+(1.0 - beta1)* (gradients['W'+str(i)])
        mb['b'+str(i)]= beta1*mb['b'+str(i)]+(1.0 - beta1)* (gradients['b'+str(i)])
        
        prevW['W'+str(i)]= beta2*prevW['W'+str(i)]+(1.0 - beta2)* (gradients['W'+str(i)]**2)
        prevb['b'+str(i)]= beta2*prevb['b'+str(i)]+(1.0 - beta2)* (gradients['b'+str(i)]**2)

        mWHat=(1.0-(beta1**f))*mW['W'+str(i)]
        mbHat=(1.0-(beta1**f))*mb['b'+str(i)]

        vWHat=(1.0-(beta2**f))*prevW['W'+str(i)]
        vbHat=(1.0-(beta2**f))*prevb['b'+str(i)]
        
        mbarW= beta1*mWHat+(1.0-beta1)*gradients['W'+str(i)]
        mbarb= beta1*mbHat+(1.0-beta1)*gradients['b'+str(i)]

        parameters['W'+str(i)] -=  ((eta/np.sqrt(vWHat+epsilon))*mbarW)
        parameters['b'+str(i)] -=  ((eta/np.sqrt(vbHat+epsilon))*mbarb)
      mini+=1
    _,_,yhat=forwardPropagation(X_train,parameters)
    print("loss after epoch ",Loss(yhat,y_train,X_train))
    
    t+=1
nadam()

Epoch  0
loss after epoch  65691.24430290182
Epoch  1
loss after epoch  56191.56502188217
Epoch  2
loss after epoch  44798.44039205763
Epoch  3
loss after epoch  40620.18673769921
Epoch  4
loss after epoch  38342.535301488264
Epoch  5
loss after epoch  36133.78848749312
Epoch  6
loss after epoch  33816.36261855071
Epoch  7
loss after epoch  32288.854075853724
Epoch  8
loss after epoch  31161.612311450663
Epoch  9
loss after epoch  30301.327255245636
Epoch  10
loss after epoch  29598.33189718381
Epoch  11
loss after epoch  29011.85155684418
Epoch  12
loss after epoch  28489.76537472226
Epoch  13
loss after epoch  27994.386162485807
Epoch  14
loss after epoch  27552.16427285564
Epoch  15
loss after epoch  27130.39653252381
Epoch  16
loss after epoch  26729.488187380473
Epoch  17
loss after epoch  26360.351182323997
Epoch  18
loss after epoch  26020.42627632413
Epoch  19
loss after epoch  25699.979826872903
Epoch  20
loss after epoch  25402.369181332084
Epoch  21
loss after epoch  25114.4

In [26]:
def predict(X_test,y_test):
  _,_,y_hat=forwardPropagation(X_test,parameters)
  y_hat=y_hat.argmax(axis=0)
  correctPred=np.sum(y_hat==y_test)
  print("Accuracy is ", correctPred/X_test.shape[0]*100,"%")
  return (y_hat,correctPred)
print(predict(X_train,y_train))
print(predict(X_test,y_test))

Accuracy is  91.24833333333333 %
(array([9, 0, 0, ..., 3, 0, 5]), 54749)
Accuracy is  86.19 %
(array([9, 2, 1, ..., 8, 1, 5]), 8619)
